In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import librosa
import os
import nltk
from nltk.corpus import stopwords

In [2]:
import librosa.display
from pydub import AudioSegment
import speech_recognition as sr
from textblob import TextBlob

In [3]:
import os
import librosa
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [4]:
# Step 1: Load the audio files and create a DataFrame
audio_dir = 'audio'  # Replace with the path to your audio folder
audio_files = os.listdir(audio_dir)
audio_df = pd.DataFrame({'filename': audio_files, 'filepath': [os.path.join(audio_dir, f) for f in audio_files]})

In [14]:
print(audio_files)

['af', 'ar', 'bn', 'bs', 'ca', 'cs', 'cy', 'da', 'de', 'desktop.ini', 'el', 'en', 'en-au', 'en-ca', 'en-gb', 'en-gh', 'en-ie', 'en-in', 'en-ng', 'en-nz', 'en-ph', 'en-tz', 'en-uk', 'en-us', 'en-za', 'eo', 'es', 'es-es', 'es-us', 'et', 'fi', 'fr', 'fr-ca', 'fr-fr', 'hi', 'hr', 'hu', 'hy', 'id', 'is', 'it', 'ja', 'jw', 'km', 'ko', 'la', 'lv', 'mk', 'ml', 'mr', 'my', 'ne', 'nl', 'no', 'pl', 'pt', 'pt-br', 'pt-pt', 'ro', 'ru', 'si', 'sk', 'sq', 'sr', 'su', 'sv', 'sw', 'ta', 'te', 'th', 'tl', 'tr', 'uk', 'vi', 'zh-cn', 'zh-tw']


In [5]:
audio_df

,filename,filepath
0,af,audio\af
1,ar,audio\ar
2,bn,audio\bn
3,bs,audio\bs
4,ca,audio\ca
...,...,...
71,tr,audio\tr
72,uk,audio\uk
73,vi,audio\vi
74,zh-cn,audio\zh-cn


In [28]:
import os
import librosa
import pandas as pd

# Define the directory where your audio files are located
audio_dir = 'audio'

# Create a list of all audio file paths in the directory
audio_files = [os.path.join(audio_dir, f) for f in os.listdir(audio_dir) if f.endswith('.mp3')]

# Define a function to extract acoustic features
def extract_acoustic_features(audio_file_path):
    y, sr = librosa.load(audio_file_path, sr=None)
    chroma_stft_mean = librosa.feature.chroma_stft(y=y, sr=sr).mean()
    rmse = librosa.feature.rms(y=y)
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y)
    return chroma_stft_mean, rmse[0][0], spectral_centroid[0][0], spectral_bandwidth[0][0], rolloff[0][0], zero_crossing_rate[0][0]




In [23]:
audio_dir = 'audio'  # Replace with the path to your audio folder
print(os.listdir())
audio_files = os.listdir(audio_dir)
audio_df = pd.DataFrame({'filename': audio_files, 'filepath': [os.path.join(audio_dir, f) for f in audio_files]})


['.ipynb_checkpoints', 'Advanced ML Final Project Code.ipynb', 'audio', 'audio_features.csv', 'HateSpeechDetectionText_EPOCH 1.ipynb', 'HateSpeechDetection_TextDataset.ipynb', 'Hate_Speech_Detection_AudioTAPAD_dataset-20.03.2023.ipynb', 'Hate_Speech_Detection_AudioTAPAD_dataset.ipynb', 'Sid.ipynb', 'Untitled.ipynb', 'Untitled1.ipynb']


In [24]:
import os

audio_dir = "./audio"
print(os.listdir(audio_dir))


['af', 'ar', 'bn', 'bs', 'ca', 'cs', 'cy', 'da', 'de', 'desktop.ini', 'el', 'en', 'en-au', 'en-ca', 'en-gb', 'en-gh', 'en-ie', 'en-in', 'en-ng', 'en-nz', 'en-ph', 'en-tz', 'en-uk', 'en-us', 'en-za', 'eo', 'es', 'es-es', 'es-us', 'et', 'fi', 'fr', 'fr-ca', 'fr-fr', 'hi', 'hr', 'hu', 'hy', 'id', 'is', 'it', 'ja', 'jw', 'km', 'ko', 'la', 'lv', 'mk', 'ml', 'mr', 'my', 'ne', 'nl', 'no', 'pl', 'pt', 'pt-br', 'pt-pt', 'ro', 'ru', 'si', 'sk', 'sq', 'sr', 'su', 'sv', 'sw', 'ta', 'te', 'th', 'tl', 'tr', 'uk', 'vi', 'zh-cn', 'zh-tw']


In [27]:
# Create a list of all audio file paths in the "audio" folder and its subdirectories
audio_files = []
for root, dirs, files in os.walk('audio'):
    for file in files:
        if file.endswith('.mp3'):
            audio_files.append(os.path.join(root, file))

# Create a DataFrame with the audio file names and their acoustic features
audio_data = {'filename': [], 'filepath': [], 'chroma_stft_mean': [], 'rmse': [], 'spectral_centroid': [], 'spectral_bandwidth': [], 'rolloff': [], 'zero_crossing_rate': []}
for audio_file in audio_files:
    audio_data['filename'].append(os.path.basename(audio_file))
    audio_data['filepath'].append(audio_file)
    features = extract_acoustic_features(audio_file)
    for i, feature_name in enumerate(['chroma_stft_mean', 'rmse', 'spectral_centroid', 'spectral_bandwidth', 'rolloff', 'zero_crossing_rate']):
        audio_data[feature_name].append(features[i])

acoustic_df = pd.DataFrame(audio_data)
print(acoustic_df)

             filename                   filepath  chroma_stft_mean      rmse  \
0            anus.mp3          audio\af\anus.mp3          0.461504  0.000172   
1            arse.mp3          audio\af\arse.mp3          0.448611  0.000000   
2        arsehole.mp3      audio\af\arsehole.mp3          0.470297  0.000000   
3         ass-hat.mp3       audio\af\ass-hat.mp3          0.450537  0.000161   
4      ass-jabber.mp3    audio\af\ass-jabber.mp3          0.442178  0.000168   
...               ...                        ...               ...       ...   
26020     wetback.mp3    audio\zh-tw\wetback.mp3          0.310399  0.000008   
26021       whore.mp3      audio\zh-tw\whore.mp3          0.430026  0.000013   
26022    whorebag.mp3   audio\zh-tw\whorebag.mp3          0.313372  0.000007   
26023   whoreface.mp3  audio\zh-tw\whoreface.mp3          0.268046  0.000007   
26024         wop.mp3        audio\zh-tw\wop.mp3          0.356107  0.000012   

       spectral_centroid  spectral_band

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm



In [60]:
import os
import speech_recognition as sr
import spacy
import subprocess


# Define a function to extract text from audio files
def extract_text(audio_path):
    # Convert audio to WAV format using FFmpeg
    wav_path = os.path.splitext(audio_path)[0] + ".wav"
    subprocess.run(['ffmpeg', '-i', audio_path, '-vn', '-acodec', 'pcm_s16le', '-ac', '1', '-ar', '16000', wav_path], check=True)
    
    # Transcribe audio to text using Google Speech Recognition
    r = sr.Recognizer()
    with sr.AudioFile(wav_path) as source:
        audio = r.record(source)
    text = r.recognize_google(audio)
    
    # Remove the temporary WAV file
    os.remove(wav_path)
    
    return text


In [61]:
def extract_linguistic_features(text):
    nlp = spacy.load('en_core_web_sm')
    
    # Use spaCy to extract linguistic features from the text
    doc = nlp(text)
    features = {
        "num_sentences": len(list(doc.sents)),
        "num_words": len(doc),
        "num_unique_words": len(set([token.text.lower() for token in doc if token.is_alpha])),
        "avg_word_length": sum([len(token.text) for token in doc if token.is_alpha]) / len([token for token in doc if token.is_alpha]),
        "num_stopwords": len([token for token in doc if token.is_stop]),
        "avg_sentence_length": len(doc) / len(list(doc.sents)),
        "num_nouns": len([token for token in doc if token.pos_ == "NOUN"]),
        "num_verbs": len([token for token in doc if token.pos_ == "VERB"]),
        "num_adjectives": len([token for token in doc if token.pos_ == "ADJ"]),
        "num_adverbs": len([token for token in doc if token.pos_ == "ADV"]),
        "num_pronouns": len([token for token in doc if token.pos_ == "PRON"]),
        "num_proper_nouns": len([token for token in doc if token.pos_ == "PROPN"]),
        "num_conjunctions": len([token for token in doc if token.pos_ == "CCONJ" or token.pos_ == "SCONJ"]),
        "num_numerals": len([token for token in doc if token.pos_ == "NUM"]),
        "num_particles": len([token for token in doc if token.pos_ == "PART"]),
        "num_punctuations": len([token for token in doc if token.pos_ == "PUNCT"]),
        "num_symbols": len([token for token in doc if token.pos_ == "SYM"])
    }
    
    return features

In [72]:
# Set the path to your audio folder
audio_folder = 'audio'

# Create an empty DataFrame to store the linguistic features for each file
#linguistic_features_df = pd.DataFrame(columns=['file_name', 'noun_chunks', 'num_sentences', 'num_tokens', 'avg_token_length', 'avg_sentence_length', 'num_stopwords', 'num_punctuations', 'num_entities'])

# Loop over each subdirectory in the audio folder
for root, dirs, files in os.walk(audio_folder):
    for file in files:
        if file.endswith('.mp3'):
            file_path = os.path.join(root, file)
            file_path = file_path.replace("\\", "/") # Replace any backslashes with forward slashes
            # Extract the text from the mp3 file
            text = extract_text(file_path)
            # Extract the linguistic features from the text
            features = extract_linguistic_features(text)
            # Add the file name to the features DataFrame
            features['file_name'] = file
            # Append the features to the DataFrame
            linguistic_features_df = linguistic_features_df.append(features, ignore_index=True)

            print(file_path)
            print(text)
# Save the DataFrame to a CSV file
#linguistic_features_df.to_csv('linguistic_features.csv', index=False)



CalledProcessError: Command '['ffmpeg', '-i', 'audio/af/anus.mp3', '-vn', '-acodec', 'pcm_s16le', '-ac', '1', '-ar', '16000', 'audio/af/anus.wav']' returned non-zero exit status 1.

In [74]:
import os
import pandas as pd
import speech_recognition as sr
import spacy

# Set the path to your audio folder
audio_folder = 'audio'

# Create an empty DataFrame to store the linguistic features for each file
linguistic_features_df = pd.DataFrame(columns=['file_name', 'noun_chunks', 'num_sentences', 'num_tokens', 'avg_token_length', 'avg_sentence_length', 'num_stopwords', 'num_punctuations', 'num_entities'])

# Loop over each subdirectory in the audio folder
for root, dirs, files in os.walk(audio_folder):
    for file in files:
        if file.endswith('.mp3'):
            file_path = os.path.join(root, file)
            file_path = file_path.replace("\\", "/") # Replace any backslashes with forward slashes
            # Convert the mp3 file to a WAV file
            wav_path = os.path.join(root, os.path.splitext(file)[0] + '.wav')
            os.system(f'ffmpeg -i "{file_path}" -vn -acodec pcm_s16le -ac 1 -ar 16000 "{wav_path}"')
            # Extract the text from the WAV file
            r = sr.Recognizer()
            with sr.AudioFile(wav_path) as source:
                audio = r.record(source)
            text = r.recognize_google(audio)
            # Extract the linguistic features from the text
            nlp = spacy.load('en_core_web_sm')
            doc = nlp(text)
            noun_chunks = [chunk.text for chunk in doc.noun_chunks]
            num_sentences = len(list(doc.sents))
            num_tokens = len(doc)
            avg_token_length = sum(len(token.text) for token in doc) / num_tokens
            avg_sentence_length = num_tokens / num_sentences
            num_stopwords = len([token for token in doc if token.is_stop])
            num_punctuations = len([token for token in doc if token.is_punct])
            num_entities = len(doc.ents)
            # Add the file name and linguistic features to the DataFrame
            features = {
                'file_name': file,
                'noun_chunks': noun_chunks,
                'num_sentences': num_sentences,
                'num_tokens': num_tokens,
                'avg_token_length': avg_token_length,
                'avg_sentence_length': avg_sentence_length,
                'num_stopwords': num_stopwords,
                'num_punctuations': num_punctuations,
                'num_entities': num_entities
            }
            linguistic_features_df = linguistic_features_df.append(features, ignore_index=True)

# Save the DataFrame to a CSV file
linguistic_features_df.to_csv('linguistic_features.csv', index=False)


UnknownValueError: 

In [75]:
import speech_recognition as sr

def extract_text_from_mp3(mp3_path):
    # Initialize a recognizer object
    r = sr.Recognizer()
    
    # Load the audio file and transcribe the speech to text
    with sr.AudioFile(mp3_path) as source:
        audio_data = r.record(source)
        text = r.recognize_google(audio_data)
    
    return text


In [ ]:
import os

# Define the root folder
root_folder = 'audio'

# Define a list to store the extracted texts
texts = []

# Traverse the folder structure and extract text from all MP3 files
for dirpath, dirnames, filenames in os.walk(root_folder):
    for filename in filenames:
        if filename.endswith('.mp3'):
            # Extract the text from the MP3 file
            mp3_path = os.path.join(dirpath, filename)
            text = extract_text_from_mp3(mp3_path)
            texts.append(text)

# Print the extracted texts
for text in texts:
    print(text)


In [98]:
import os
import subprocess
import speech_recognition as sr

def extract_text(audio_path):
    # Convert audio file to WAV format
    if audio_path.endswith('.mp3'):
        wav_path = os.path.splitext(audio_path)[0] + '.wav'
        subprocess.call(['ffmpeg', '-i', audio_path, '-vn', '-acodec', 'pcm_s16le', '-ac', '1', '-ar', '16000', wav_path])
    else:
        wav_path = audio_path
        # Transcribe audio to text
    r = sr.Recognizer()
    with sr.AudioFile(wav_path) as source:
        audio = r.record(source)
    text = r.recognize_google(audio)
    return text
path = r"C:\Users\siddh\MY PYTHON\Machine_Learning_Practice_2023\MachineLearning\AML_Project\audio\af"
extract_text(path)

def extract_linguistic_features(text):
    nlp = spacy.load('en_core_web_sm')
    
    # Use spaCy to extract linguistic features from the text
    doc = nlp(text)
    features = {
        "num_sentences": len(list(doc.sents)),
        "num_words": len(doc),
        "num_unique_words": len(set([token.text.lower() for token in doc if token.is_alpha])),
        "avg_word_length": sum([len(token.text) for token in doc if token.is_alpha]) / len([token for token in doc if token.is_alpha]),
        "num_stopwords": len([token for token in doc if token.is_stop]),
        "avg_sentence_length": len(doc) / len(list(doc.sents)),
        "num_nouns": len([token for token in doc if token.pos_ == "NOUN"]),
        "num_verbs": len([token for token in doc if token.pos_ == "VERB"]),
        "num_adjectives": len([token for token in doc if token.pos_ == "ADJ"]),
        "num_adverbs": len([token for token in doc if token.pos_ == "ADV"]),
        "num_pronouns": len([token for token in doc if token.pos_ == "PRON"]),
        "num_proper_nouns": len([token for token in doc if token.pos_ == "PROPN"]),
        "num_conjunctions": len([token for token in doc if token.pos_ == "CCONJ" or token.pos_ == "SCONJ"]),
        "num_numerals": len([token for token in doc if token.pos_ == "NUM"]),
        "num_particles": len([token for token in doc if token.pos_ == "PART"]),
        "num_punctuations": len([token for token in doc if token.pos_ == "PUNCT"]),
        "num_symbols": len([token for token in doc if token.pos_ == "SYM"])
    }
    
    return features

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\siddh\\MY PYTHON\\Machine_Learning_Practice_2023\\MachineLearning\\AML_Project\\audio\\af'

In [100]:
import os
import subprocess
import speech_recognition as sr
import spacy

def extract_text(audio_path):
    # Convert audio file to WAV format
    if audio_path.endswith('.mp3'):
        wav_path = os.path.splitext(audio_path)[0] + '.wav'
        subprocess.call(['ffmpeg', '-i', audio_path, '-vn', '-acodec', 'pcm_s16le', '-ac', '1', '-ar', '16000', wav_path])
    else:
        wav_path = audio_path
    
    # Transcribe audio to text
    r = sr.Recognizer()
    with sr.AudioFile(wav_path) as source:
        audio = r.record(source)
    text = r.recognize_google(audio)
    return text

def extract_linguistic_features(text):
    nlp = spacy.load('en_core_web_sm')
    
    # Use spaCy to extract linguistic features from the text
    doc = nlp(text)
    features = {
        "num_sentences": len(list(doc.sents)),
        "num_words": len(doc),
        "num_unique_words": len(set([token.text.lower() for token in doc if token.is_alpha])),
        "avg_word_length": sum([len(token.text) for token in doc if token.is_alpha]) / len([token for token in doc if token.is_alpha]),
        "num_stopwords": len([token for token in doc if token.is_stop]),
        "avg_sentence_length": len(doc) / len(list(doc.sents)),
        "num_nouns": len([token for token in doc if token.pos_ == "NOUN"]),
        "num_verbs": len([token for token in doc if token.pos_ == "VERB"]),
        "num_adjectives": len([token for token in doc if token.pos_ == "ADJ"]),
        "num_adverbs": len([token for token in doc if token.pos_ == "ADV"]),
        "num_pronouns": len([token for token in doc if token.pos_ == "PRON"]),
        "num_proper_nouns": len([token for token in doc if token.pos_ == "PROPN"]),
        "num_conjunctions": len([token for token in doc if token.pos_ == "CCONJ" or token.pos_ == "SCONJ"]),
        "num_numerals": len([token for token in doc if token.pos_ == "NUM"]),
        "num_particles": len([token for token in doc if token.pos_ == "PART"]),
        "num_punctuations": len([token for token in doc if token.pos_ == "PUNCT"]),
        "num_symbols": len([token for token in doc if token.pos_ == "SYM"])
    }
    
    return features

# Traverse through all subdirectories of "audio" folder
for root, dirs, files in os.walk("audio"):
    for file in files:
        if file.endswith(".mp3"):
            audio_path = os.path.join(root, file)
            print(f"Processing file: {audio_path}")
            
            # Convert audio to text and extract features
            try:
                text = extract_text(audio_path)
                features = extract_linguistic_features(text)
                print(f"Transcribed text: {text}")
                print(f"Extracted features: {features}")
            except Exception as e:
                print(f"Error processing file {audio_path}: {e}")


Error processing file audio\af\kooch.mp3: 
Processing file: audio\af\kootch.mp3
Error processing file audio\af\kootch.mp3: 
Processing file: audio\af\kraut.mp3
Error processing file audio\af\kraut.mp3: 
Processing file: audio\af\kunt.mp3
Error processing file audio\af\kunt.mp3: 
Processing file: audio\af\kyke.mp3
Error processing file audio\af\kyke.mp3: 
Processing file: audio\af\lameass.mp3
Error processing file audio\af\lameass.mp3: 
Processing file: audio\af\lardass.mp3
Error processing file audio\af\lardass.mp3: 
Processing file: audio\af\lesbian.mp3
Error processing file audio\af\lesbian.mp3: 
Processing file: audio\af\lesbo.mp3
Error processing file audio\af\lesbo.mp3: 
Processing file: audio\af\lezzie.mp3
Error processing file audio\af\lezzie.mp3: 
Processing file: audio\af\mcfagget.mp3
Error processing file audio\af\mcfagget.mp3: 
Processing file: audio\af\mick.mp3
Error processing file audio\af\mick.mp3: 
Processing file: audio\af\minge.mp3
Error processing file audio\af\minge

KeyboardInterrupt: 